# 1인가구수 집계 데이터 수집 및 전처리

- 시군구 이름 통합 기준
  - 소계(전국), 수도권, 충청권, 호남권, 대경권, 동남권, 강원권


---


  - 전국 -> 다 더하기
  - 소계 -> 제주, 세종 ??
  - 읍부, 면부, 동부 -> 삭제
  - 서울특별시 + 인천광역시 + 경기도 -> 수도권 (통합)
  - 충청북도 + 충청남도 + 대전광역시, 세종특별자치시 -> 충청권 (통합)
  - 전라북도 + 전라남도 -> 호남권 (통합)
  - 대구광역시 + 경상북도 -> 대경권 (통합)
  - 부산광역시 + 울산광역시 + 경상남도 -> 동남권 (통합)

  - **제주특별자치도 -> 처리?**


## 모듈 설치

In [11]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.25.2', '1.5.3')

## 구글 드라이브 연동

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 준비 및 전처리

### 권별로 통합

In [13]:
member_1 = pd.read_csv('/content/drive/MyDrive/멋사/Team Space/data/raw_data/1인가구.csv') # 데이터 불러오기

member_2 = member_1.iloc[:, [0, 1, 2, 10, 18, 26, 34]] # 행정구역별(시군구), 성별, 2018, 2019, 2020, 2021, 2022 필요한 열만 남기기
drop_index_list = list(range(4, 13)) # 필요없는 행 삭제(읍부, 면부, 동부 삭제)
member_2 = member_2.drop(drop_index_list)
member_2 = member_2.T.reset_index()

member_2.iloc[0] = member_2.iloc[0].astype(str) + '_' + member_2.iloc[1].astype(str) # 첫번쨰, 두번째 행 합치
member_2 = member_2.drop(1)

member_2.columns = member_2.loc[0]
member_2 = member_2.drop(member_2.index[0])
member_2.columns.values[0] = 'year'
member_2.columns.values[1] = '1인가구'

for col in member_2.columns[2:]: # 2번쨰 열('소계')부터
    member_2[col] = pd.to_numeric(member_2[col]) # float 형식으로 변환

regions = {
    '수도권': ['서울특별시', '인천광역시', '경기도'],
    '충청권': ['충청북도', '충청남도', '대전광역시', '세종특별자치시'],
    '호남권' : ['전라북도', '전라남도'],
    '대경권' : ['대구광역시', '경상북도'],
    '동남권' : ['부산광역시','울산광역시','경상남도'],
    '제주특별자치도' : ['제주특별자치도']
}
words = ['계', '여자', '남자']

# 반복문을 사용하여 각 지역에 대한 열 추가
for region, region_cities in regions.items():
    for word in words:
        columns_to_sum = [f'{city}_{word}' for city in region_cities]
        member_2[f'{region}_{word}'] = member_2[columns_to_sum].sum(axis=1)

member_2 = member_2.drop(member_2.columns[5:-18], axis=1)

cols_to_move = member_2.columns[5:8] # 제주도 계/남자/여자 -> 제일 마지막 열로 재배치
temp_df = member_2[cols_to_move]
member_2 = member_2.drop(cols_to_move, axis=1)
member_2 = pd.concat([member_2, temp_df], axis=1)
member_2
# # member_2.to_csv('1인가구수_권별.csv', encoding='cp949', index=False)

,year,1인가구,전국_계,전국_남자,전국_여자,수도권_계,수도권_여자,수도권_남자,충청권_계,충청권_여자,...,호남권_남자,대경권_계,대경권_여자,대경권_남자,동남권_계,동남권_여자,동남권_남자,제주특별자치도_계,제주특별자치도_남자,제주특별자치도_여자
2,2018,1인가구,5848594,2906320,2942274,2702905,1330075,1372830,705913,341263,...,215734,623605,330000,293605,894363,464206,430157,73250,37607,35643
3,2019,1인가구,6147516,3053733,3093783,2888545,1425653,1462892,740212,356737,...,221816,644629,339931,304698,930663,483302,447361,75353,38296,37057
4,2020,1인가구,6643354,3304398,3338956,3121552,1543802,1577750,813600,392417,...,241796,693334,364255,329079,995792,515340,480452,81855,41643,40212
5,2021,1인가구,7165788,3583770,3582018,3388650,1668783,1719867,866321,414542,...,262146,743563,387193,356370,1070913,551323,519590,88683,44806,43877
6,2022,1인가구,7502350,3751071,3751279,3574726,1766192,1808534,901256,429280,...,271824,772636,400608,372028,1117613,575066,542547,92172,46304,45868


In [14]:
member_1

,행정구역별(시군구),성별,2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,...,2021.6,2021.7,2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7
0,행정구역별(시군구),성별,1인가구,주택_계,주택_단독주택,주택_아파트,주택_연립주택,주택_다세대주택,주택_비거주용건물내 주택,주택이외의 거처_계,...,주택_비거주용건물내 주택,주택이외의 거처_계,1인가구,주택_계,주택_단독주택,주택_아파트,주택_연립주택,주택_다세대주택,주택_비거주용건물내 주택,주택이외의 거처_계
1,전국,계,5848594,5269889,2762080,1750760,98323,538112,120614,578705,...,132304,809379,7502350,6640775,3075767,2553695,134435,743570,133308,861575
2,전국,남자,2906320,2562867,1439547,755843,42371,257199,67907,343453,...,75792,462516,3751071,3266732,1658546,1113655,59873,358870,75788,484339
3,전국,여자,2942274,2707022,1322533,994917,55952,280913,52707,235252,...,56512,346863,3751279,3374043,1417221,1440040,74562,384700,57520,377236
4,읍부,계,539706,500727,272158,175509,14770,29059,9231,38979,...,10582,48648,694083,643773,313243,261788,20334,37818,10590,50310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,경상남도,남자,185174,168619,109114,50070,2179,3843,3413,16555,...,3681,20753,233713,212394,126633,74070,3019,4917,3755,21319
60,경상남도,여자,194545,185229,112667,62536,3072,4212,2742,9316,...,2968,12004,235059,222528,118668,91388,4041,5423,3008,12531
61,제주특별자치도,계,73250,64093,37297,13905,4441,6428,2022,9157,...,2213,12596,92172,79247,42942,18831,6593,8610,2271,12925
62,제주특별자치도,남자,37607,32019,19379,6096,2089,3318,1137,5588,...,1276,7025,46304,39196,22310,8230,3054,4313,1289,7108


### 시별로 년도별 계만

In [10]:
member_1.columns

Index(['행정구역별(시군구)', '성별', '2018', '2018.1', '2018.2', '2018.3', '2018.4',
       '2018.5', '2018.6', '2018.7', '2019', '2019.1', '2019.2', '2019.3',
       '2019.4', '2019.5', '2019.6', '2019.7', '2020', '2020.1', '2020.2',
       '2020.3', '2020.4', '2020.5', '2020.6', '2020.7', '2021', '2021.1',
       '2021.2', '2021.3', '2021.4', '2021.5', '2021.6', '2021.7', '2022',
       '2022.1', '2022.2', '2022.3', '2022.4', '2022.5', '2022.6', '2022.7'],
      dtype='object')

In [15]:
member_1 = pd.read_csv('/content/drive/MyDrive/멋사/Team Space/data/raw_data/1인가구.csv') # 데이터 불러오기
member_3 = member_1.iloc[:, [0, 2, 10, 18, 26, 34]] # 행정구역별(시군구), 성별, 2018, 2019, 2020, 2021, 2022
member_3 = member_3.loc[[13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61]]
member_3.columns.values[0] = 'year'

for col in member_3.columns[3:]: # 2번쨰 열부터
    member_3[col] = pd.to_numeric(member_3[col]) # float 형식으로 변환

new_columns = {col: int(col) for col in member_3.columns[1:5]}  # 두 번째부터 다섯 번째 열 이름을 정수형으로 변경
member_3 = member_3.rename(columns=new_columns)
member_3
# member_3.to_csv('1인가구수_시별.csv', encoding='cp949', index=False)

,year,2018,2019,2020,2021,2022
13,서울특별시,1229421,1299787,1390701,1489893,1564187
16,부산광역시,404408,423227,455207,487362,511745
19,대구광역시,269903,284416,304543,326866,341667
22,인천광역시,275898,297865,324841,355657,376392
25,광주광역시,174614,182364,193948,212385,221464
28,대전광역시,195544,205165,228842,240932,249006
31,울산광역시,110236,116015,122848,133061,137096
34,세종특별자치시,35763,39078,43577,45706,50561
37,경기도,1197586,1290893,1406010,1543100,1634147
40,강원도,206295,208857,231371,245012,254441


In [16]:
member_3

(17, 6)